In [3]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score

In [4]:
data = pd.read_csv('./BANCO CRED CREA Despesas_Classificadas_Codificadas.csv', sep=',')

In [5]:
data.head()

,Despesa,Categoria,Categoria_Codificada
0,pg internet white martins,Despesas Diversas,2
1,pg internet med curitiba,Saúde,9
2,pg internet springer carrier,Equipamentos,5
3,credito ted pagseguro internet ltda,Seguros,10
4,pg internet aluguel,Aluguel,0


In [6]:
X_train, X_test, y_train, y_test = train_test_split(data['Despesa'], data['Categoria_Codificada'], test_size=0.2, random_state=42)

In [7]:
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

In [20]:
rf_model = RandomForestClassifier(random_state=42, n_estimators=500)
rf_model.fit(X_train_tfidf, y_train)

RandomForestClassifier(n_estimators=500, random_state=42)

### Aplicando o modelo direto (sem balanceamento)

In [21]:
y_pred = rf_model.predict(X_test_tfidf)

In [22]:
present_labels = sorted(set(y_test))  # Identificar as classes presentes no conjunto de teste
report = classification_report(y_test, y_pred, labels=present_labels)

d:\Repositories\AVD\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Repositories\AVD\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Repositories\AVD\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [23]:
accuracy = accuracy_score(y_test, y_pred)
print(f"Acurácia: {accuracy * 100:.2f}%\n")
print("Relatório de Classificação:\n", report)

Acurácia: 75.00%

Relatório de Classificação:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00         1
           1       1.00      0.50      0.67         2
           2       0.60      1.00      0.75         3
           4       1.00      1.00      1.00         1
           6       0.00      0.00      0.00         1

    accuracy                           0.75         8
   macro avg       0.72      0.70      0.68         8
weighted avg       0.72      0.75      0.70         8



### Modelo com ROS (balanceamento de classes)

In [31]:
from imblearn.over_sampling import RandomOverSampler

**Sobre o ROS:**

O Random Forest utiliza o conceito de aprendizado em conjunto (ensemble learning), onde múltiplos modelos (árvores de decisão, no caso) trabalham em conjunto para produzir um resultado mais robusto.

- Cada árvore é construída a partir de uma amostra aleatória dos dados (técnica de bootstrap).

- As previsões de todas as árvores são combinadas para determinar o resultado final:

- **Classificação:** A classe final é escolhida com base no voto majoritário entre as árvores.

- **Regressão:** A previsão final é a média dos valores previstos pelas árvores.

In [ ]:
X = data['Despesa'] # Features
y = data['Categoria_Codificada'] # Labels

vectorizer = TfidfVectorizer()
X_tfidf = vectorizer.fit_transform(X)

In [ ]:
# Instanciar o RandomOverSampler para lidar com o desbalanceamento

ros = RandomOverSampler(random_state=42)
X_resampled, y_resampled = ros.fit_resample(X_tfidf, y)

In [ ]:
# Dividir o conjunto de dados em treino e teste

X_train, X_test, y_train, y_test = train_test_split(X_resampled, y_resampled, test_size=0.2, random_state=42)

In [ ]:
# Treinar o modelo Random Forest com os dados balanceados

rf_model = RandomForestClassifier(random_state=42, n_estimators=200, max_depth=15, min_samples_split=5)
rf_model.fit(X_train, y_train)

RandomForestClassifier(max_depth=15, min_samples_split=5, n_estimators=200,
                       random_state=42)

In [ ]:
# Avaliação do modelo e relatório de classificação

y_pred = rf_model.predict(X_test)

accuracy = accuracy_score(y_test, y_pred)
report = classification_report(y_test, y_pred)

print(f"Acurácia: {accuracy * 100:.2f}%\n")
print("Relatório de Classificação:\n", report)

Acurácia: 92.59%

Relatório de Classificação:
               precision    recall  f1-score   support

           0       1.00      1.00      1.00         5
           1       0.60      1.00      0.75         3
           2       0.00      0.00      0.00         1
           3       1.00      1.00      1.00         3
           4       1.00      1.00      1.00         2
           5       1.00      1.00      1.00         1
           6       1.00      0.75      0.86         4
           7       1.00      1.00      1.00         1
           8       1.00      1.00      1.00         2
           9       1.00      1.00      1.00         1
          10       1.00      1.00      1.00         1
          11       1.00      1.00      1.00         3

    accuracy                           0.93        27
   macro avg       0.88      0.90      0.88        27
weighted avg       0.92      0.93      0.91        27



d:\Repositories\AVD\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Repositories\AVD\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Repositories\AVD\venv\Lib\site-packages\sklearn\metrics\_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [53]:
# Testando o modelo com novas despesas e exibindo as categorias preditas

novas_despesas = ["pg internet celular", "credito ted pic pay","pg internet net trabalho","pg internet net casa", "db allianz seguro carro"]
novas_despesas_tfidf = vectorizer.transform(novas_despesas)
novas_predicoes = rf_model.predict(novas_despesas_tfidf)


encoder = LabelEncoder()
encoder.fit(data['Categoria_Codificada'])

for despesa, categoria in zip(novas_despesas, encoder.inverse_transform(novas_predicoes)):
    print(f"Despesa: {despesa} -> Categoria Predita: {categoria}")

Despesa: pg internet celular -> Categoria Predita: 1
Despesa: credito ted pic pay -> Categoria Predita: 8
Despesa: pg internet net trabalho -> Categoria Predita: 1
Despesa: pg internet net casa -> Categoria Predita: 1
Despesa: db allianz seguro carro -> Categoria Predita: 10


In [ ]:
# Adicionar uma coluna com as predições no dataframe

data['Categoria_Preditada'] = rf_model.predict(X_tfidf)
data.to_csv('./BANCO CRED CREA Despesas_Classificadas_Codificadas.csv', index=False)